In [1]:
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass  # To get the password without showing the input

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#sns.set_theme(style="whitegrid")

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
password = getpass("INSERT YOUR PASSWORD: ")

INSERT YOUR PASSWORD: ········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
data1 = pd.read_sql_query('SELECT * FROM sakila.rentals_may', engine)
data1.head()

,customer_id,rental_date
0,130,2005-05-24 22:53:30
1,459,2005-05-24 22:54:33
2,408,2005-05-24 23:03:39
3,333,2005-05-24 23:04:41
4,222,2005-05-24 23:05:21


In [5]:
data2 = pd.read_sql_query('SELECT * FROM sakila.rentals_june', engine)
data2.head()

,customer_id,rental_date
0,416,2005-06-14 22:53:33
1,516,2005-06-14 22:55:13
2,239,2005-06-14 23:00:34
3,285,2005-06-14 23:07:08
4,310,2005-06-14 23:09:38


In [6]:
#Check the number of rentals for each customer for May
data1.value_counts()

customer_id  rental_date        
1            2005-05-25 11:30:37    1
367          2005-05-31 11:38:29    1
390          2005-05-30 10:58:33    1
             2005-05-26 14:43:48    1
389          2005-05-31 00:16:57    1
                                   ..
197          2005-05-27 11:03:55    1
             2005-05-26 09:02:47    1
             2005-05-25 16:03:42    1
196          2005-05-31 07:12:44    1
599          2005-05-31 01:18:56    1
Length: 1156, dtype: int64

In [7]:
#Check the number of rentals for each customer for June
data2.value_counts()

customer_id  rental_date        
1            2005-06-15 00:54:12    1
365          2005-06-15 09:55:57    1
387          2005-06-20 08:35:17    1
             2005-06-21 13:51:28    1
388          2005-06-15 08:00:13    1
                                   ..
197          2005-06-15 14:05:11    1
             2005-06-15 22:07:09    1
             2005-06-16 06:17:55    1
             2005-06-17 05:40:52    1
599          2005-06-21 18:46:05    1
Length: 2311, dtype: int64

In [8]:
#Check the number of rentals for each customer for May
query = 'SELECT customer_id, COUNT(*) AS "number of rentals_may" FROM rentals_may GROUP BY customer_id;'

rentals_may_df = pd.read_sql_query(query, engine)
rentals_may_df.head()

,customer_id,number of rentals_may
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [9]:
len(rentals_may_df)

520

In [10]:
#Check the number of rentals for each customer for June
query = 'SELECT customer_id, COUNT(*) AS "number of rentals_june" FROM rentals_june GROUP BY customer_id;'

rentals_june_df = pd.read_sql_query(query, engine)
rentals_june_df.head()

,customer_id,number of rentals_june
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


In [11]:
len(rentals_june_df)

590

In [12]:
df1 = rentals_may_df.merge(rentals_june_df, how = 'inner', on = 'customer_id')

In [13]:
df1.head()

,customer_id,number of rentals_may,number of rentals_june
0,130,2,6
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2


In [ ]:
len(outer_df)

In [ ]:
#Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
#inner join both tables from Mayn and June
df = pd.merge(left=rentals_may_df, right=rentals_june_df, left_on='customer_id', right_on='customer_id')
df.shape
df

In [14]:
#rename the columns
df1 = df1.rename(columns={"number of rentals_may":"num_rentals_may","number of rentals_june":"num_rentlas_june"})
df1.columns

Index(['customer_id', 'num_rentals_may', 'num_rentlas_june'], dtype='object')

In [ ]:
#apply conditions to see depending on customer_id, which month had the higher rent using apply lambda. However, I couldn't include more than one condition
df1['Higher_rental'] = df1.apply(lambda x: 'june' if x['num_rentals_may'] <
                     x['num_rentlas_june'] else 'may', axis=1)

In [15]:
df1

,customer_id,num_rentals_may,num_rentlas_june
0,130,2,6
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2
...,...,...,...
507,431,1,7
508,191,2,6
509,351,1,3
510,10,1,5


In [ ]:
# I tried to define this function to be able to deefine more conditions. However, I got error all the time. I couldn't solve it!
def max_rental(df):
    if (df["num_rentals_may"] > df["num_rentals_june"]):
        return "May"
    elif (df["num_rentals_may"] < df["num_rentals_june"]):
        return "June"
    else:
        return "equal"
    
df["max"] = df.apply(max_rental, axis=1)

In [ ]:
df['max'] = np.where(df['num_rentals_june']<df['num_rentals_may'], 'May',
                   np.where(df['num_rentals_june']>df['num_rentals_may'], 'june',
                   np.where(df['num_rentals_june']==df['num_rentals_may'], 'equal')))

In [ ]:
df['max'] = (np.where(df['num_rentals_june']<df['num_rentals_may'] , df['max'].apply(lambda x: 'may'),
                       np.where(df.apply(lambda x: 'june' if x['num_rentals_may'] <
                     x['num_rentlas_june'] else 'equal', axis=1))

In [ ]:
df1

In [16]:
df1['May_June'] = 'More rented June'

In [19]:
df1

,customer_id,num_rentals_may,num_rentlas_june,May_June
0,130,2,6,More rented June
1,459,1,7,More rented June
2,408,3,3,More rented June
3,333,1,4,More rented June
4,222,5,2,More rented June
...,...,...,...,...
507,431,1,7,More rented June
508,191,2,6,More rented June
509,351,1,3,More rented June
510,10,1,5,More rented June


In [17]:
df1['May_June'][df1['num_rentals_june']<df1['num_rentals_may']] = 'more rented May'

KeyError: 'num_rentals_june'

In [18]:
df1['May_June'][df1['num_rentals_june'] == df1['num_rentals_may']] = 'same'

KeyError: 'May-June'